In [ ]:
#나눔폰트 설

In [ ]:
#sudo apt-get update

In [ ]:
#sudo apt-get install -y fonts-nanum

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import logging

logging.getLogger("matplotlib.font_manager").setLevel(logging.ERROR)

fontpath = "/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf"
fontprop = fm.FontProperties(fname=fontpath, size=12)
plt.rcParams["font.family"] = fontprop.get_name()

print(f"설정된 폰트: {fontprop.get_name()}")

설정된 폰트: NanumBarunGothic


In [3]:
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.2 MB/s eta 0:00:00a 0:00:01


In [4]:
import os
import re
import urllib.request
import zipfile
import sentencepiece as spm
import pandas as pd

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)

2.7.1+cu118


In [6]:
%cd ~/work/s2s_translation  

/home/jovyan/work/s2s_translation


In [13]:
# 1. tar 명령어로 압축을 풉니다.
# 현재 폴더에 있는 파일을 지정하므로, 앞의 경로 부분은 모두 제거합니다.
!tar -xzvf korean-english-park.train.tar.gz

# 2. 압축 해제 후 파일 목록을 확인합니다.
# korean-english-park.train.tsv 파일이 새로 생성된 것을 볼 수 있습니다.
!ls -l

korean-english-park.train.en
korean-english-park.train.ko
total 33972
-rw-r--r-- 1 jovyan users   113461 Sep  3 16:04 korean-english-park.dev.tar.gz
-rw-r--r-- 1 jovyan users   229831 Sep  3 16:04 korean-english-park.test.tar.gz
-rw-r--r-- 1 jovyan users 11982221 Jul 12  2014 korean-english-park.train.en
-rw-r--r-- 1 jovyan users 13730884 Jul 12  2014 korean-english-park.train.ko
-rw-r--r-- 1 jovyan users  8718893 Sep  3 16:04 korean-english-park.train.tar.gz


In [34]:
# ======================================================================
# ## 1단계: 데이터 로드 및 통합
# # 로컬에 저장된 tar.gz 파일들의 압축을 풀고,
# # 분리된 한국어(.ko), 영어(.en) 파일을 읽어 하나의 DataFrame으로 만듭니다.
# ======================================================================
print("--- 1단계: 데이터 로드 시작 ---")

# 데이터 파일 목록을 딕셔너리로 정의합니다.
file_list = {
    'train': 'korean-english-park.train.tar.gz',
    'dev': 'korean-english-park.dev.tar.gz',
    'test': 'korean-english-park.test.tar.gz'
}

# 생성된 데이터프레임을 저장할 딕셔너리를 초기화합니다.
data_frames = {}

# 각 파일을 순회하며 압축을 풀고 DataFrame을 생성합니다.
for name, file_name in file_list.items():
    # 1. tar.gz 압축 해제
    with tarfile.open(file_name, 'r:gz') as tar:
        tar.extractall()
        
    # 2. 압축 해제된 파일들의 경로를 정의합니다.
    base_name = file_name.replace(".tar.gz", "")
    ko_file_path = f"{base_name}.ko"
    en_file_path = f"{base_name}.en"
    
    # 3. 한국어 파일을 한 줄씩 읽어 리스트에 저장합니다.
    with open(ko_file_path, 'r', encoding='utf-8') as f:
        ko_lines = [line.strip() for line in f]
    
    # 4. 영어 파일을 한 줄씩 읽어 리스트에 저장합니다.
    with open(en_file_path, 'r', encoding='utf-8') as f:
        en_lines = [line.strip() for line in f]
        
    # 5. DataFrame 생성 (첫 줄은 파일명이므로 제외)
    df = pd.DataFrame({
        'korean': ko_lines[1:],
        'english': en_lines[1:]
    })
    
    # 6. 생성된 DataFrame을 딕셔너리에 저장합니다.
    data_frames[name] = df

# 각 데이터프레임을 사용하기 쉬운 변수에 할당합니다.
train_df = data_frames['train']
dev_df = data_frames['dev']
test_df = data_frames['test']

print(" 모든 데이터셋(train, dev, test) 준비 완료!")
# --- [확인] 로드된 데이터 샘플 출력 ---
pd.set_option('display.max_colwidth', None)
print("\n--- [확인] 로드된 Train 데이터 샘플 (상위 3개) ---")
print(train_df.head(3))
# --- [추가] 각 데이터셋의 개수 확인 ---
print(f"\nTrain 데이터 개수: {len(train_df)}")
print(f"Dev 데이터 개수: {len(dev_df)}")
print(f"Test 데이터 개수: {len(test_df)}")
print("="*50)



--- 1단계: 데이터 로드 시작 ---


/tmp/ipykernel_518/13032387.py:22: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall()


✅ 모든 데이터셋(train, dev, test) 준비 완료!

--- [확인] 로드된 Train 데이터 샘플 (상위 3개) ---
                                                                                       korean  \
0                                        모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.   
1                                                                  그러나 이것은 또한 책상도 필요로 하지 않는다.   
2  79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분이든 그 움직임에따라 커서의 움직임을 조절하는 회전 운동 센서를 사용하고 있다.   

                                                                                                                     english  
0  so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.  
1                                                                    Like all optical mice, But it also doesn't need a desk.  
2              uses gyroscopic sensors to control the cursor movement as you move your wrist, arm, whatever through the air.  

Train 데이터 개수: 94122
Dev 데이터 

* 데이터가 .. 퀄이 좋지 않음
* 우선 다음 작업으로 넘어가 보다... 한사이클 돌려보자

In [36]:

# ======================================================================
# ## 2단계: 텍스트 전처리
# # 문장을 모델이 학습하기 좋은 형태로 정제합니다.
# # 숫자 내 쉼표(,) 제거, 문장 부호 분리 등을 수행합니다.
# ======================================================================
print("\n--- 2단계: 텍스트 전처리 함수 정의 및 테스트 ---")

def preprocess_sentence(sentence):
    sentence = str(sentence).lower().strip()
    sentence = re.sub(r"(\d),(\d)", r"\1\2", sentence)
    sentence = re.sub(r"([?.!])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z가-힣0-9?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

# --- [확인] 전처리 함수 테스트 ---
sample_idx = 100
original_sentence = train_df['korean'][sample_idx]
preprocessed_sentence = preprocess_sentence(original_sentence)
print("\n--- [확인] 전처리 함수 적용 결과 ---")
print("원본 문장:", original_sentence)
print("전처리 후:", preprocessed_sentence)
print("="*50)




--- 2단계: 텍스트 전처리 함수 정의 및 테스트 ---

--- [확인] 전처리 함수 적용 결과 ---
원본 문장: 1,960만갤론 이상의 중유를 운반하던 손상된 유조선이 스페인 북서부 연안 130마일 해역에서 침몰하여, 스페인 당국이 어업이 주요 산업인 이 지역에서 해안선 보호를 위해 황급히 서두르고 있다.
전처리 후: 1960만갤론 이상의 중유를 운반하던 손상된 유조선이 스페인 북서부 연안 130마일 해역에서 침몰하여, 스페인 당국이 어업이 주요 산업인 이 지역에서 해안선 보호를 위해 황급히 서두르고 있다 .


* 전처리 수정 기록
1차 숫자가 제거됨 > 2차 숫자사이 쉼표로 숫자가 분리됨 1 , 960 > 3차 숫자 사이 쉼표 별도 제거

In [38]:
# ======================================================================
# ## 3단계: SentencePiece 토크나이저 훈련
# # 훈련(Train) 데이터만을 사용하여 토크나이저 모델을 생성합니다.
# ======================================================================
print("\n--- 3단계: SentencePiece 토크나이저 훈련 시작 ---")

with open("sp_train_ko.txt", "w", encoding="utf-8") as f:
    for sentence in train_df['korean']:
        f.write(preprocess_sentence(sentence) + '\n')

with open("sp_train_en.txt", "w", encoding="utf-8") as f:
    for sentence in train_df['english']:
        f.write(preprocess_sentence(sentence) + '\n')

vocab_size = 8000
pad_id, bos_id, eos_id, unk_id = 0, 1, 2, 3

spm.SentencePieceTrainer.train(
    f'--input=sp_train_ko.txt --model_prefix=korean_spm --vocab_size={vocab_size} '
    f'--pad_id={pad_id} --bos_id={bos_id} --eos_id={eos_id} --unk_id={unk_id} --model_type=bpe'
)
spm.SentencePieceTrainer.train(
    f'--input=sp_train_en.txt --model_prefix=english_spm --vocab_size={vocab_size} '
    f'--pad_id={pad_id} --bos_id={bos_id} --eos_id={eos_id} --unk_id={unk_id} --model_type=bpe'
)

print(" 토크나이저 훈련 완료!")
# --- [수정] 생성된 단어 사전(.vocab) 내용을 파이썬으로 직접 읽어 확인 ---
print("\n--- [확인] 생성된 한국어 단어 사전 (상위 10개 토큰) ---")
with open('korean_spm.vocab', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 10:
            break
        print(line.strip())
print("="*50)




--- 3단계: SentencePiece 토크나이저 훈련 시작 ---
✅ 토크나이저 훈련 완료!

--- [확인] 생성된 한국어 단어 사전 (상위 10개 토큰) ---
<pad>	0
<s>	0
</s>	0
<unk>	0
▁.	-0
▁이	-1
했다	-2
▁있	-3
에서	-4
▁대	-5


In [39]:
# ======================================================================
# ## 4단계: 전체 데이터셋 토큰화 및 후처리
# # 훈련된 토크나이저를 사용하여 모든 데이터(train, dev, test)를
# # 숫자 시퀀스로 변환하고, 후처리 작업을 수행합니다.
# ======================================================================
print("\n--- 4단계: 전체 데이터셋 토큰화 및 후처리 시작 ---")

sp_ko = spm.SentencePieceProcessor(); sp_ko.load('korean_spm.model')
sp_en = spm.SentencePieceProcessor(); sp_en.load('english_spm.model')

for df in [train_df, dev_df, test_df]:
    df['ko_tokens'] = df['korean'].apply(lambda x: [bos_id] + sp_ko.encode_as_ids(preprocess_sentence(x)) + [eos_id])
    df['en_tokens'] = df['english'].apply(lambda x: [bos_id] + sp_en.encode_as_ids(preprocess_sentence(x)) + [eos_id])

print(" 모든 데이터셋 토큰화 및 BOS/EOS 추가 완료!")

# --- [확인] 토큰화 및 BOS/EOS 추가 결과 ---
print("\n--- [확인] 토큰화 적용 결과 (첫 번째 샘플) ---")
sample_row = train_df.iloc[0]
print("Original Korean:", sample_row['korean'])
print("Tokenized Korean:", sample_row['ko_tokens'])
print("Original English:", sample_row['english'])
print("Tokenized English:", sample_row['en_tokens'])
print("="*50)

print("\n--- 길이 필터링 시작 ---")
max_len = 50
def filter_by_length(df, max_len):
    before_len = len(df)
    df = df[df.apply(lambda x: len(x['ko_tokens']) <= max_len and len(x['en_tokens']) <= max_len, axis=1)]
    after_len = len(df)
    print(f"길이 필터링 전: {before_len} -> 후: {after_len} ({(before_len - after_len):,}개 제거)")
    return df

train_df = filter_by_length(train_df, max_len)
dev_df = filter_by_length(dev_df, max_len)
test_df = filter_by_length(test_df, max_len)
print("="*50)

print("\n--- 최종 결과 확인 (필터링 후 Train 데이터 샘플) ---")
print(train_df.head())



--- 4단계: 전체 데이터셋 토큰화 및 후처리 시작 ---
✅ 모든 데이터셋 토큰화 및 BOS/EOS 추가 완료!

--- [확인] 토큰화 적용 결과 (첫 번째 샘플) ---
Original Korean: 모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.
Tokenized Korean: [1, 439, 714, 6881, 6910, 1660, 5581, 172, 5, 714, 6881, 3452, 6851, 863, 6863, 5797, 1261, 6818, 73, 3452, 675, 1948, 546, 6826, 1043, 1324, 4, 2]
Original English: so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.
Tokenized English: [1, 377, 5, 5312, 5, 1316, 1498, 1162, 255, 5, 50, 124, 30, 237, 535, 7457, 3190, 339, 25, 566, 2625, 28, 869, 50, 124, 30, 237, 535, 7983, 7457, 25, 566, 12, 2]

--- 길이 필터링 시작 ---
길이 필터링 전: 94122 -> 후: 83804 (10,318개 제거)
길이 필터링 전: 999 -> 후: 865 (134개 제거)
길이 필터링 전: 1999 -> 후: 1732 (267개 제거)

--- 최종 결과 확인 (필터링 후 Train 데이터 샘플) ---
                                                                                              korean  \
0                                               모든 광마우스와 마찬가지 로 이 광마우스

In [40]:
# ======================================================================
# ## 5단계: 라이브러리 임포트 및 기본 설정
# ======================================================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import random
import math
import time

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [41]:
# ======================================================================
# ## 6단계: 데이터셋 및 데이터로더 생성
# # pandas DataFrame을 PyTorch가 사용할 수 있는 Dataset과 DataLoader로 변환합니다.
# # DataLoader는 학습 시 데이터를 배치(batch) 단위로 묶어주는 역할을 합니다.
# ======================================================================

# PyTorch Dataset 클래스 정의
class TextDataset(Dataset):
    def __init__(self, df):
        self.src_sents = df['ko_tokens'].tolist()
        self.trg_sents = df['en_tokens'].tolist()

    def __len__(self):
        return len(self.src_sents)

    def __getitem__(self, idx):
        return torch.tensor(self.src_sents[idx]), torch.tensor(self.trg_sents[idx])

# collate_fn: DataLoader가 배치를 만들 때, 문장의 길이를 맞추기 위해 패딩(padding)을 적용합니다.
def collate_fn(batch):
    src_batch, trg_batch = [], []
    for src_sample, trg_sample in batch:
        src_batch.append(src_sample)
        trg_batch.append(trg_sample)
    
    # pad_sequence를 사용하여 PAD_ID(0)로 길이를 맞춥니다.
    src_padded = pad_sequence(src_batch, batch_first=True, padding_value=pad_id)
    trg_padded = pad_sequence(trg_batch, batch_first=True, padding_value=pad_id)
    return src_padded, trg_padded

# Dataset 인스턴스 생성
train_dataset = TextDataset(train_df)
dev_dataset = TextDataset(dev_df)
test_dataset = TextDataset(test_df)

# DataLoader 생성
BATCH_SIZE = 128
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
dev_iterator = DataLoader(dev_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

print("\n DataLoader 준비 완료!")
print(f"Train 배치 개수: {len(train_iterator)}")



 DataLoader 준비 완료!
Train 배치 개수: 655


In [42]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()

        self.W1 = nn.Linear(hidden_dim, hidden_dim)
        self.W2 = nn.Linear(hidden_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden: (batch_size, hidden_dim)
        # encoder_outputs: (src_len, batch_size, hidden_dim)

        src_len = encoder_outputs.shape[0]

        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)  # (batch_size, src_len, hidden_dim)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)  # (batch_size, src_len, hidden_dim)

        energy = torch.tanh(self.W1(encoder_outputs) + self.W2(hidden))  # (batch_size, src_len, hidden_dim)
        attention = self.v(energy).squeeze(2)  # (batch_size, src_len)

        return nn.functional.softmax(attention, dim=1)  # (batch_size, src_len)

In [43]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim)

    def forward(self, src):
        # src : (src_len, batch_size)
        embedded = self.embedding(src)  # embedded : (src_len, batch_size, emb_dim)
        outputs, hidden = self.rnn(embedded)  # outputs : (src_len, batch_size, hidden_dim)

        return outputs, hidden

In [44]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, attention):
        super(Decoder, self).__init__()

        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        # Decoder RNN에는 embedding만 입력
        self.rnn = nn.GRU(emb_dim, hidden_dim)
        # 출력층에는 hidden state와 attention value가 결합되어 입력
        self.fc_out = nn.Linear(hidden_dim + hidden_dim, output_dim)

    def forward(self, input, hidden, encoder_outputs):
        # input : (batch_size,)
        # hidden : (batch_size, hidden_dim)
        # encoder_outputs : (src_len, batch_size, hidden_dim)
        input = input.unsqueeze(0)  # input : (1, batch_size)
        embedded = self.embedding(input)  # embedded : (1, batch_size, emb_dim)

        # attention distribution을 계산합니다. decoder의 이전 hidden state, s_{t-1}와 encoder의 H가 입력됩니다.
        a = self.attention(hidden[-1], encoder_outputs)  # a : (batch_size, src_len)

        # H에 가중치를 부여해 attention value(Context vector) 계산
        a = a.unsqueeze(1)  # a : (batch_size, 1, src_len)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)  # encoder_outputs : (batch_size, src_len, hidden_dim)
        context = torch.bmm(a, encoder_outputs)  # context : (batch_size, 1, hidden_dim)
        context = context.permute(1, 0, 2)  # context : (1, batch_size, hidden_dim)

        output, hidden = self.rnn(embedded, hidden)

        # 출력층에서는 현재 hidden state와 context vector를 결합하여 예측값 생성
        output = output.squeeze(0)  # output : (batch_size, hidden_dim)
        context = context.squeeze(0)  # context : (batch_size, hidden_dim)
        prediction = self.fc_out(torch.cat((output, context), dim=1))  # (batch_size, output_dim)

        return prediction, hidden, a.squeeze(1)

In [45]:
class Seq2SeqAttention(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg=None, max_len=30, bos_id = 1, eos_id=2):
        # 학습 모드에서는 trg_len 사용, 추론 모드에서는 max_len까지 동적 생성
        batch_size = src.shape[1]
        trg_vocab_size = self.decoder.fc_out.out_features

        # 조기 종료를 위해 tensor가 아닌 리스트 사용
        outputs = []

        # 시각화를 위해 attention 저장
        attentions = []

        # 인코더를 통해 context 생성
        encoder_outputs, hidden = self.encoder(src)

        if trg is not None:
            for t in range(0, trg.shape[0]):
                input = trg[t]
                output, hidden, attention = self.decoder(input, hidden, encoder_outputs)
                outputs.append(output.unsqueeze(0))
                attentions.append(attention.unsqueeze(0))

        else:
		    # inference에서는 target(정답)이 없기 때문에 sos_token을 생성해줍니다.
            input = torch.full((batch_size,), bos_id, dtype=torch.long, device=self.device)
            finished = torch.zeros(batch_size, dtype=torch.bool, device=self.device)

            for t in range(max_len):
                output, hidden, attention = self.decoder(input, hidden,  encoder_outputs)
                outputs.append(output.unsqueeze(0))
                attentions.append(attention.unsqueeze(0))
                top1 = output.argmax(1)
                input = top1

                # 조기 종료 조건
                finished |= (top1 == eos_id)
                if finished.all():
                    break

        outputs = torch.cat(outputs, dim=0)  # (trg_len, batch_size, output_dim)
        attentions = torch.cat(attentions, dim=0)  # (trg_len, batch_size, src_len)

        return outputs, attentions

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_dim = len(encoder_tokenizer)
output_dim = len(decoder_tokenizer)encoder_tokenizer
emb_dim = 256
hid_dim = 512

NameError: name 'encoder_tokenizer' is not defined